In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision import transforms

from tqdm import tqdm
from PIL import Image

import glob

from efficientnet_pytorch import EfficientNet
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import os
import boto3
import cv2

torch.cuda.empty_cache()

In [2]:
class GLDV2(Dataset):

    def __init__(self, stage: str, inferance=True):
        self.stage = stage
        self.df = read_csv(stage)
        self.df.drop(self.df.filter(regex="Unname"), axis=1, inplace=True)
        self.label_list = self.df.landmark_id.tolist()
        self.namelist = [i.split('\\')[-1] for i in self.df.anchor.tolist()]
        print(f'shape of df is {len(self.df)}, stage is {stage}')
        self.s3path = 'Your data set path'
        self.my_transformer = transforms.Compose([
          transforms.Resize((224,224)),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])

    def __getitem__(self, index):
        label = self.df.iloc[index]['landmark_id']
        name = self.namelist[index]
        anchor = self.df.iloc[index]['anchor'].split('\\')
        anchor_class = anchor[1]
        anchor_filen = anchor[2].split('.')[0].lower()
        if self.stage == 'index':     
            anchor_image = '/home/bo/work/tianyi/dataset_original/train/' + name[0] +'/' + name[1] +'/' + name[2] +'/' +anchor_filen+'.jpg'
        else:
            anchor_image = '/home/bo/work/tianyi/dataset_original/test/' + anchor_filen+'.jpg'
        anchor_im = Image.open(anchor_image).convert('RGB')
#         anchor_im = cv2.imread(anchor_image)
#         anchor_im = cv2.cvtColor(anchor_im, cv2.COLOR_BGR2RGB)
        transformed_anchor_im = self.my_transformer(anchor_im)
        return transformed_anchor_im 

    def __len__(self):
        return len(self.df)


def read_csv(stage):

    if stage == 'public':
        df = pd.read_csv('Your public csv path')
    elif stage == 'private':
        df = pd.read_csv('Your private csv path')
    elif stage == 'public_private':
        df = pd.read_csv('Your public_private csv path')
    elif stage == 'index':
        df = pd.read_csv('Your index csv path')

    else:
        raise ValueError(f'not supported stage{stage}')
    return df

In [3]:
def extract_global_features(model, dataloader, stage = 'index'):
    with torch.inference_mode():
        BATCH_IDX = 0
        for batch_idx, batch_img in enumerate(dataloader):
            print(f'current batch is {batch_idx} at {stage}')
            embed = model.extract_features(batch_img.to(device))
            embed = embed.cpu().detach().numpy()
            print(embed.shape)
            # embed = embed.reshape(500,1280,7,7)
            embed_np = np.mean(embed,axis=(2,3))
            with open(SAVE_PATH + stage + '/' + str(BATCH_IDX).zfill(3) + '.npy', 'wb') as f:
                np.save(f, np.array(embed_np))
            BATCH_IDX += 1


SAVE_PATH = 'out/'

device = 'cuda'

query_dataset = GLDV2('public_private')
index_dataset = GLDV2('index')

query_dataloader = DataLoader(query_dataset, batch_size=500, shuffle=False)
index_dataloader = DataLoader(index_dataset, batch_size=500, shuffle=False)


embed_model = EfficientNet.from_pretrained('efficientnet-b0',num_classes=10)
model_path = 'model path'
model_state = torch.load(model_path)['model']
embed_model.load_state_dict(model_state)
embed_model.cuda()

embed_model.eval()
# extract_global_features(embed_model, index_dataloader, 'index')
extract_global_features(embed_model, query_dataloader, 'test')

shape of df is 1000, stage is public_private
shape of df is 100000, stage is index
Loaded pretrained weights for efficientnet-b0
current batch is 0 at test
(500, 1280, 7, 7)
current batch is 1 at test
(500, 1280, 7, 7)
